In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [4]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, count, udf, sort_array
from pyspark.sql import Window
import pyspark.sql.types as t

import itertools

def pairs(a):
    return filter(lambda (x,y): x<y, itertools.product(a, repeat=2))

pairs_udf = udf(pairs, t.ArrayType(t.StructType([
    t.StructField("user1", t.IntegerType(), True), 
    t.StructField("user2", t.IntegerType(), True)
])))

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .filter(size(col("users")) > 1) \
    .select(sort_array("users").alias("sorted_users")) \
    .withColumn("user", explode(pairs_udf('sorted_users'))) \
    .groupBy(col("user.user1"), col("user.user2")) \
    .agg(count("*").alias("friends_count"))

In [5]:
reversedGraph.printSchema()

root
 |-- user1: integer (nullable = true)
 |-- user2: integer (nullable = true)
 |-- friends_count: long (nullable = false)



In [ ]:
window = Window.orderBy(col("friends_count").desc())
    
top50 = reversedGraph.withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .select(col("friends_count"), col("user1"), col("user2")) \
            .orderBy(col("friends_count").desc()) \
            .collect()

In [ ]:
for val in top50:
    print '%s %s %s' % val